# Obtaining Our Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

Your boss gives you a general description of some of the datasets at your disposal for analyzing weekly store sales. They're eventually looking for you to build a model to help determine what factors impact sales, and model future sales forecasting for business planning.  
  
Most of the properietary store data sits in the company sql database, accessible by all managers and above. The database is called **Walmart.db** Your boss provides you with the following basic schema:  

<img src='db_schema.jpg' width=500>  

She then tells you that she's put together a second dataset on general economy statistics for the various dates that she would also like you to incorporate in your analysis. That data, she says, is stored in a file **economy_data.csv**.

As a first step in creating your model for providing recommendations and projections, load and synthesize these disperate datasets into a singular unified DataFrame. Then save your results to a file **Merged_Store_Data.csv**.

Make sure you check the various data types and merge appropriately.

In [2]:
# imports
import sqlite3
import pandas as pd
import matplotlib as plt

In [4]:
# sql data to a dataframe
con = sqlite3.connect('Walmart.db')
cur = con.cursor()
cur.execute("""select * from sales join store_details using (store);""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.head(10)

(452192, 7)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315
2,1,1,2010-02-19,41595.55,False,A,151315
3,1,1,2010-02-26,19403.54,False,A,151315
4,1,1,2010-03-05,21827.90,False,A,151315
5,1,1,2010-03-12,21043.39,False,A,151315
6,1,1,2010-03-19,22136.64,False,A,151315
7,1,1,2010-03-26,26229.21,False,A,151315
8,1,1,2010-04-02,57258.43,False,A,151315
9,1,1,2010-04-09,42960.91,False,A,151315


In [6]:
econ_df = pd.read_csv('economy_data.csv')
print(econ_df.shape)
econ_df.head(10)

(8190, 12)


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
5,1,2010-03-12,57.79,2.667,NaN,NaN,NaN,NaN,NaN,211.380643,8.106,False
6,1,2010-03-19,54.58,2.720,NaN,NaN,NaN,NaN,NaN,211.215635,8.106,False
7,1,2010-03-26,51.45,2.732,NaN,NaN,NaN,NaN,NaN,211.018042,8.106,False
8,1,2010-04-02,62.27,2.719,NaN,NaN,NaN,NaN,NaN,210.820450,7.808,False
9,1,2010-04-09,65.86,2.770,NaN,NaN,NaN,NaN,NaN,210.622857,7.808,False


In [ ]:
# there appears to be NaN values in the MarkDown columns.
# Also, it's not clear what is meant by IsHoliday because each record is a week.
# Common columns are Store, Date, and IsHoliday

In [8]:
matched_cols = ['Store', 'Date', 'IsHoliday']
for col in matched_cols:
    ex1 = df[col].iloc[0]
    ex2 = econ_df[col].iloc[0]
    print(col)
    print('Types:')
    print('df1: {}, df2: {}'.format(type(ex1), type(ex2)))
    print('\n')

Store
Types:
df1: <class 'numpy.int64'>, df2: <class 'numpy.int64'>


Date
Types:
df1: <class 'str'>, df2: <class 'str'>


IsHoliday
Types:
df1: <class 'str'>, df2: <class 'numpy.bool_'>




In [9]:
# IsHoliday is a string in one dataframe, and a bool in the other
df.IsHoliday = df.IsHoliday.astype(bool)

In [10]:
# Merge the dataframes
merged = pd.merge(df, econ_df)
print(merged.shape)
merged.head(10)

(31817, 16)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
1,1,2,2010-02-12,44682.74,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
2,1,3,2010-02-12,10887.84,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
3,1,4,2010-02-12,35351.21,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
4,1,5,2010-02-12,29620.81,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
5,1,6,2010-02-12,9135.00,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
6,1,7,2010-02-12,18310.31,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
7,1,8,2010-02-12,37334.83,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
8,1,9,2010-02-12,16562.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106
9,1,10,2010-02-12,31494.77,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.24217,8.106


In [11]:
merged.to_csv('Merged_Store_Data.csv', index=False)

## Summary
Nice work! You're working more and more independently through the workflow, and ensuring data integrity!